In [1]:
# the copywright author is added, which is notebook specific
# for more information on codeowners and details, check CODEOWNERS
__copyright__ = "Copywright © 2025 Debmalya Pramanik (ZenithClown)"

<div align = "center">

# Basic Concepts & Usage of `NetworkX`

</div>

[**`NetworkX`**](https://networkx.org/) is a Python library for creating, manipulating, and studying the structure, dynamics, and functions of complex networks. It provides tools for working with graphs, which are mathematical structures used to model relationships between entities.This document captures the usage of ``routicle`` along side the base module in terms of supply chain management problems. For more information on graph theory and usage of NetworkX check the detailed [documentation](https://networkx.org/documentation/stable/tutorial.html).

## Installation & Importing NetworkX

Check [PyPI Package](https://pypi.org/project/networkx/) for more information on installation and requirements. If NetworkX is not already installed, it can be installed using pip:

```shell
$ pip install networkx
```

It is standard practice to import NetworkX with the alias `nx`. The module provides different graph classes, `nx.Graph` for undirected graph, and `nx.DiGraph` for directed ones.

```python
import networkx as nx

# create a undirected graph, with alias (say G)
G = nx.Graph()

# or, create a directed graph, with alias (say DG)
DG = nx.DiGraph()
```

For fundamental concepts on adding graph components like `nodes` and `edges` and any basic information check [tutorial](https://networkx.org/documentation/stable/tutorial.html).

In [2]:
import os  # miscellaneous os interfaces
import sys # configuring python runtime environment

In [3]:
import json # pretty print dictionary object

In [4]:
import random # generate random numbers for poc development

In [5]:
from typing import Iterable
from uuid import uuid4 as UUIDx
from tqdm.auto import tqdm as TQ

### Software for Complex Network

**`NetworkX`** is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks. In addition, we are using **`gravis`** which is great for visualization of graphs and has an inbuilt feature of conversion of a primitive `nx.Graph` to a [D3.js](https://d3js.org/) visualization for export/view inline in jupyter notebook.

In [6]:
import gravis as gv
import networkx as nx

## User Defined Function(s)

It is recommended that any UDFs are defined outside the scope of the *jupyter notebook* such that development/editing of function can be done more practically. As per *programming guidelines* as [`src`](https://fileinfo.com/extension/src) file/directory is beneficial in code development and/or production release. However, *jupyter notebook* requires *kernel restart* if any imported code file is changed in disc, for this frequently changing functions can be defined in this section.

**Getting Started** with **`PYTHONPATH`**

One must know what are [Environment Variable](https://medium.com/chingu/an-introduction-to-environment-variables-and-how-to-use-them-f602f66d15fa) and how to call/use them in your choice of programming language. Note that an environment variable is *case sensitive* in all operating systems (except windows, since DOS is not case sensitive). Generally, we can access environment variables from terminal/shell/command prompt as:

```shell
# macOS/*nix
echo $VARNAME

# windows
echo %VARNAME%
```

Once you've setup your system with [`PYTHONPATH`](https://bic-berkeley.github.io/psych-214-fall-2016/using_pythonpath.html) as per [*python documentation*](https://docs.python.org/3/using/cmdline.html#envvar-PYTHONPATH) is an important directory where any `import` statements looks for based on their order of importance. If a source code/module is not available check necessary environment variables and/or ask the administrator for the source files.

Most of the utility functions available in `PYTHONPATH` is tracked and maintained in [GIST.GitHub/ZenithClown](https://gist.github.com/ZenithClown) which provides detailed documentation and code snippets/example use cases etc. For more information, and category wise module check [github](https://github.com/ZenithClown/ZenithClown) repository.

In [7]:
sys.path.append(os.path.join("..")) # base of the repository

# methods/class objects to define graph components
from routicle.components import edges, nodes
from routicle.utils.generator import create_nodes

# methods/class objects for core functionalities of scm & logistics applications
import routicle.core.networkx as rnx

In [8]:
def unique_keys(*args : Iterable[dict]) -> bool:
    ndicts = len(args) # number of dictionary to validate

    keys = []
    for di in TQ(args, desc = "Getting All Keys"):
        keys += list(di.keys())

    return len(keys) == len(set(keys))

## Global Argument(s)

The global arguments are *notebook* specific, however they may also be extended to external libraries and functions on import. The *boilerplate* provides a basic ML directory structure which contains a directory for `data` and a separate directory for `output`. In addition, a separate directory (`data/processed`) is created to save processed dataset such that preprocessing can be avoided.

### Graph Components

In graph theory, a network is a structure consisting of nodes (or vertices) representing objects and edges (or links) representing connections or relationships between them. These abstract structures are used to model and analyze real-world systems, such as the Internet, social relationships, and biological processes. Networks can be undirected (where edges are bidirectional) or directed (where edges have a specific direction), and can also be weighted with numerical values to represent the strength or cost of the connection.

The module focuses on supply chain problem - like share of business optimization, inventory planning, etc. using different graph theory and shortest path detection algorithm like `Dijkstra/A*` which are the core components. A component is always an instance of `instance(<object>, routicle.components.base.BaseComponent)` class.

In [9]:
# let's consider a network of ports, plants, vendors and warehouses
N_PORTS, N_PLANTS, N_VENDORS, N_WAREHOUSES = 2, 7, 3, 4

### Dummy Network Builder

Utility module is provided to create multiple nodes, etc. for testing and other validation purposes. Let's create a network with n-objects as defined above.

In [10]:
# node type :: ports - v0.0.1.dev.0 default constraints are capacity
ports = { node.name : node for node in list(create_nodes(N_PORTS, ntype = nodes.StorageUnits, prefix = "L")) }

# node type :: plants - v0.0.1.dev.0 constraints are rate, demand and default capacity
plants = {
    node.name : node for node in list(create_nodes(
        N_PLANTS, ntype = nodes.ManufacturingUnits, prefix = "P",
        attributes = [dict(rate = 0, demand = random.randint(100, 1000))] * N_PLANTS
    ))
}

# node type :: vendors - v0.0.1.dev.0 default constraints are capacity
vendors = {
    node.name : node for node in list(create_nodes(
        N_VENDORS, ntype = nodes.SupplyPoints, prefix = "V",
        attributes = [dict(minorder = random.randint(0, 100), maxcapacity = random.randint(500, 1000)) for _ in range(N_VENDORS)]
    ))
}

# node type :: warehouses - v0.0.1.dev.0 default constraints are capacity
warehouses = { node.name : node for node in list(create_nodes(N_WAREHOUSES, ntype = nodes.StorageUnits, prefix = "W")) }

# all nodes are packed into a single iterable json, do not overlap names
assert unique_keys(ports, plants, vendors, warehouses), "Unique Key Error in D-Nodes."
dnodes = {**ports, **plants, **vendors, **warehouses}

# all the edges are packed into a single iterable json, this should not overlap
dedges = {
    (dnodes[u].name, dnodes[v].name) : edges.TimeCostEdge(
        cidx = str(UUIDx()),
        name = f"{dnodes[u].name} --> {dnodes[v].name}",
        unode = dnodes[u], vnode = dnodes[v],
        cost = round(random.random() * random.randint(1, 10), 2),
        time = round(random.random() * random.randint(1, 10), 2)
    ) for (u, v) in [
        # edges from warehouse to plants::
        ("W0", "P1"), ("W0", "P3"), ("W0", "P6"),
        ("W1", "P1"), ("W1", "P2"), ("W1", "P3"),
        ("W2", "P2"), ("W2", "P5"),
        ("W3", "P0"), ("W3", "P3"), ("W3", "P4"), ("W3", "P5"),

        # edges from ports to warehouses::
        ("L0", "W0"), ("L0", "W1"), ("L0", "W2"),
        ("L1", "W0"), ("L1", "W1"), ("L1", "W2"), ("L1", "W3"),

        # edges from ports and plants::
        ("L0", "W0"), ("L0", "W1"), ("L0", "W2"),
        ("L1", "W0"), ("L1", "W1"), ("L1", "W2"), ("L1", "W3"),

        # edges between a vendor with different entities::
        # a local vendor can import to warehouse and a plant
        ("V0", "P0"), ("V0", "P1"), ("V0", "P3"), ("V0", "P5"),
        ("V0", "P0"), ("V0", "P1"), ("V0", "P3"), ("V0", "P5"),
        ("V1", "P0"), ("V1", "P2"), ("V1", "P3"), ("V1", "P4"), ("V1", "P6"),
        ("V1", "W0"), ("V1", "W1"), ("V1", "W2"), ("V1", "W3"),
        # a import vendor can deliver directly to a port, or others
        ("V2", "L0"), ("V2", "L1")
    ]
}

Getting All Keys:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
network = rnx.nxGraph(G = nx.DiGraph(), dnodes = dnodes, dedges = dedges)

In [13]:
fig = gv.d3(network.G, show_menu = True, show_node_image = True)
# fig.display() # or view `inline = True`

#### Feature(s) of `nxGraph`

Typical methods of graphs are wrapped inside the `rnx.nxGraph` for forward integration.

In [13]:
# inspect any graph component and return underlying dnode/dedge by ``name`` atribute, which must be defined
print("Get a dNode Object:", json.dumps(dict(network.inspect("P0")), default = str, indent = 2), end = "\n\n") # return a node, default component
print("Get a dEdge Object:", json.dumps(dict(network.inspect(("V2", "L0"), component = "edge")), default = str, indent = 2))

Get a dNode Object: {
  "name": "P0",
  "cidx": "P0",
  "image": "../assets/icons/conveyor.png",
  "mincapacity": 0.0,
  "maxcapacity": Infinity,
  "rate": 0.0,
  "demand": 529.0
}

Get a dEdge Object: {
  "name": "V2 --> L0",
  "cidx": "94d90d96-0070-458c-99a7-ada6bc99ebbf",
  "unode": "name='V2' cidx='V2' image='../assets/icons/vendor.png' mincapacity=0.0 maxcapacity=961.0 moq=0.0 packsize=1.0 reliability=1.0 minorder=28.0",
  "vnode": "name='L0' cidx='L0' image='../assets/icons/warehouse.png' mincapacity=0.0 maxcapacity=inf",
  "time": 3.1,
  "cost": 2.49
}


In [15]:
# or get the component by the ``cidx`` attribute, if defined, which is recommended to always be unique
# network.getbycidx("94d90d96-0070-458c-99a7-ada6bc99ebbf", component = "edge")

TimeCostEdge(name='V2 --> L0', cidx='94d90d96-0070-458c-99a7-ada6bc99ebbf', unode=SupplyPoints(name='V2', cidx='V2', image='../assets/icons/vendor.png', mincapacity=0.0, maxcapacity=961.0, moq=0.0, packsize=1.0, reliability=1.0, minorder=28.0), vnode=StorageUnits(name='L0', cidx='L0', image='../assets/icons/warehouse.png', mincapacity=0.0, maxcapacity=inf), time=3.1, cost=2.49)

In [16]:
# alter the graph dynamically, reverse the edge/make the graph undirected
# this returns a copy of the `network.G` attribute, without changing the underlying
rG = network.alter(reverse = True, undirected = False)

fig = gv.d3(rG, show_menu = True, show_node_image = True)
# fig.display() # or view `inline = True`

In [15]:
# easily get any adjacent nodes, for example, get vendor connections
# i.e., the points where vendor can supply in the network
network.adjacent_nodes("V1")

('P0', 'P2', 'P3', 'P4', 'P6', 'W0', 'W1', 'W2', 'W3')

In [19]:
# or, mix with graph .alter() method, for example
print(network.adjacent_nodes("P0"))

# however, we can reverse the nature to get connected points
# i.e., from where supply is available to a plant
network.adjacent_nodes("P0", reverse = True)

()


('V0', 'V1', 'W3')

### Paths Analysis

In [13]:
source, target = dnodes["V1"], dnodes["P4"]
print("All Connected Paths b/w Source & Target Node:", json.dumps(network.getpaths(source = source, target = target, roundvalue = 2), indent = 2))

All Connected Paths b/w Source & Target Node: {
  "paths": {
    "nodes": [
      [
        "V1",
        "P4"
      ],
      [
        "V1",
        "W3",
        "P4"
      ]
    ],
    "edges": [
      [
        [
          "V1",
          "P4"
        ]
      ],
      [
        [
          "V1",
          "W3"
        ],
        [
          "W3",
          "P4"
        ]
      ]
    ],
    "weights": [
      7.54,
      0.07
    ],
    "costs": [
      0.72,
      0.72
    ]
  },
  "source": "V1",
  "target": "P4"
}


In [17]:
paths = network.sortedpaths(source = source, target = target)["paths"] # or, use .getpaths() to get the raw values, as above
costs, weights = [list(map(lambda x : round(x, 2), paths[k])) for k in ["costs", "weights"]]

print(f"Found {len(costs)} Paths from {source.name} to {target.name}.")
print(f"  >> Costs (in sorted order)   : {costs}")
print(f"  >> Weights (in sorted order) : {weights}")

Found 2 Paths from V1 to P4.
  >> Costs (in sorted order)   : [0.72, 0.72]
  >> Weights (in sorted order) : [7.54, 0.07]


In [18]:
print(f"Traced Nodes   :: ", paths["nodes"])
print(f"Resolved Edges :: ", paths["edges"])

Traced Nodes   ::  [['V1', 'P4'], ['V1', 'W3', 'P4']]
Resolved Edges ::  [[('V1', 'P4')], [('V1', 'W3'), ('W3', 'P4')]]


In [20]:
# we've found the paths, we can also display the graph and highlight graph
# this is a destructive method, overrides the original graph by deleting and creating edge
flatten = [ e for edge in paths["edges"] for e in edge ]

G = network.G # create a copy of the original graph
for edge in TQ(flatten, desc = "Recreating Edges"):
    current = dedges[edge] # lookup the original value
    G.remove_edge(current.unode.name, current.vnode.name)

    # okay, we update to a random color of choice
    current._color = "#C72828"
    G.add_edge(current.unode.name, current.vnode.name, **current.attributes)

fig = gv.d3(G, show_menu = True, show_node_image = True)
# fig.display() # or view `inline = True`

Recreating Edges:   0%|          | 0/3 [00:00<?, ?it/s]